In [96]:
import pandas as pd
import numpy as np

In [97]:
FBposts = pd.read_csv("./data/facebook-fact-check.csv")

# Dates to Float  

In [98]:
FBposts_datesDF=pd.to_datetime(FBposts['Date Published'])

In [99]:
FBposts_datesfloatDF= (FBposts_datesDF - FBposts_datesDF.min())  / np.timedelta64(1,'D') 

In [72]:
type(FBposts_datesfloatDF)

pandas.core.series.Series

In [100]:
print(len(FBposts.columns))

12


In [101]:
FBposts['date_delta']=FBposts_datesfloatDF
print(len(FBposts.columns))

13


## Clean NaN

In [102]:
FBposts_cleanedNaN = FBposts.fillna(FBposts.mean())

# Dealing with Category or author from who's writing what on FB posts

In [103]:
from NLP_nltk import levenshtein_distance

In [104]:
FBposts_cleanedNaN.columns

Index([u'account_id', u'post_id', u'Category', u'Page', u'Post URL',
       u'Date Published', u'Post Type', u'Rating', u'Debate', u'share_count',
       u'reaction_count', u'comment_count', u'date_delta'],
      dtype='object')

In [105]:
FBposts_Categorynorm=pd.DataFrame([levenshtein_distance(FBposts_cleanedNaN["Category"].mode()[0],author) for author in FBposts_cleanedNaN['Category']])

In [106]:
FBposts_cleanedNaN['Categorynorm']=FBposts_Categorynorm
print(len(FBposts_cleanedNaN.columns))

14


# more data; normalizing `account_id`

In [107]:
FBposts_acctidnorm=pd.DataFrame([levenshtein_distance(str(FBposts_cleanedNaN["account_id"].mode()[0]),str(acctid)) for acctid in FBposts_cleanedNaN['account_id']])

In [108]:
FBposts_cleanedNaN['account_idnorm']=FBposts_acctidnorm
print(len(FBposts_cleanedNaN.columns))

15


In [109]:
FBpostsX=FBposts_cleanedNaN[["account_idnorm","Categorynorm","date_delta","share_count","reaction_count","comment_count"]]

## Having a $Y$ value

In [110]:
FBpostsY=FBposts_cleanedNaN["Rating"]

In [111]:
FBpostsY=FBpostsY.replace("mostly true",0.75)
FBpostsY=FBpostsY.replace("no factual content",0.5)
FBpostsY=FBpostsY.replace("mixture of true and false",0.25)
FBpostsY=FBpostsY.replace("mostly false",0.0)


In [85]:
FBpostsY.index

RangeIndex(start=0, stop=2282, step=1)

In [112]:
FBpostsX.describe()

,account_idnorm,Categorynorm,date_delta,share_count,reaction_count,comment_count
count,2282.000000,2282.000000,2282.000000,2.282000e+03,2282.000000,2282.000000
mean,8.770377,4.192375,3.816389,4.044816e+03,5364.284649,516.102193
std,4.897891,4.222367,2.857424,2.937061e+04,19118.157564,3567.790281
min,0.000000,0.000000,0.000000,1.000000e+00,2.000000,0.000000
25%,11.000000,0.000000,1.000000,2.500000e+01,149.000000,37.000000
50%,11.000000,0.000000,3.000000,1.080000e+02,548.000000,132.000000
75%,12.000000,8.000000,7.000000,1.005000e+03,2422.750000,391.000000
max,13.000000,9.000000,8.000000,1.088995e+06,456458.000000,159047.000000


# normalize further the data for account_idnorm, Categorynorm with Gaussians, share_count, reaction_count, comment_count with log

In [113]:
import sklearn
from sklearn import preprocessing

In [114]:
from sklearn.preprocessing import FunctionTransformer
transformer = FunctionTransformer(np.log1p)

In [115]:
FBpostsXnorm=FBpostsX
FBpostsXnorm["account_idnorm"]=preprocessing.scale(transformer.transform( FBpostsX['account_idnorm'].values.reshape(-1,1)))
FBpostsXnorm["Categorynorm"]  =preprocessing.scale(transformer.transform( FBpostsX['Categorynorm'].values.reshape(-1,1)))


/home/mobicfd/.local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/mobicfd/.local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [116]:
FBpostsXnorm.describe()

,account_idnorm,Categorynorm,date_delta,share_count,reaction_count,comment_count
count,2.282000e+03,2.282000e+03,2282.000000,2.282000e+03,2282.000000,2282.000000
mean,2.989137e-16,4.981895e-17,3.816389,4.044816e+03,5364.284649,516.102193
std,1.000219e+00,1.000219e+00,2.857424,2.937061e+04,19118.157564,3567.790281
min,-1.803096e+00,-9.959682e-01,0.000000,1.000000e+00,2.000000,0.000000
25%,5.195405e-01,-9.959682e-01,1.000000,2.500000e+01,149.000000,37.000000
50%,5.195405e-01,-9.959682e-01,3.000000,1.080000e+02,548.000000,132.000000
75%,5.943562e-01,9.640425e-01,7.000000,1.005000e+03,2422.750000,391.000000
max,6.636248e-01,1.058028e+00,8.000000,1.088995e+06,456458.000000,159047.000000


Now to do it for share_count, reaction_count, comment_count

In [117]:
FBpostsXnorm["share_count"]=preprocessing.scale(transformer.transform( FBpostsX['share_count'].values.reshape(-1,1)))
FBpostsXnorm["reaction_count"]  =preprocessing.scale(transformer.transform( FBpostsX['reaction_count'].values.reshape(-1,1)))
FBpostsXnorm["comment_count"]=preprocessing.scale(transformer.transform( FBpostsX['comment_count'].values.reshape(-1,1)))


/home/mobicfd/.local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/mobicfd/.local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/mobicfd/.local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [118]:
FBpostsXnorm.describe()

,account_idnorm,Categorynorm,date_delta,share_count,reaction_count,comment_count
count,2.282000e+03,2.282000e+03,2282.000000,2.282000e+03,2.282000e+03,2.282000e+03
mean,2.989137e-16,4.981895e-17,3.816389,1.619116e-16,2.740042e-16,-5.729179e-16
std,1.000219e+00,1.000219e+00,2.857424,1.000219e+00,1.000219e+00,1.000219e+00
min,-1.803096e+00,-9.959682e-01,0.000000,-1.800733e+00,-2.659409e+00,-2.881335e+00
25%,5.195405e-01,-9.959682e-01,1.000000,-7.583862e-01,-7.302285e-01,-7.023641e-01
50%,5.195405e-01,-9.959682e-01,3.000000,-1.759402e-01,-9.039570e-02,4.806048e-02
75%,5.943562e-01,9.640425e-01,7.000000,7.271964e-01,6.419058e-01,6.955441e-01
max,6.636248e-01,1.058028e+00,8.000000,3.566592e+00,3.225071e+00,4.293052e+00


# Training data, (Cross-)Validation, Test Data 

In [119]:
ratiotrain=0.85
ratiovalid=0.1
ratiotest =0.05

In [140]:
N = len(FBpostsY.index)
indshuffled=sklearn.utils.shuffle(np.arange(N))
trainind=indshuffled[:int(ratiotrain*N)]
validind=indshuffled[int(ratiotrain*N):int((ratiotrain+ratiovalid)*N)]
testind=indshuffled[int((ratiotrain+ratiovalid)*N):]
print(len(trainind))
print(len(validind))
print(len(testind))


1939
228
115


In [148]:
FB_X_train=FBpostsXnorm.iloc[trainind]
FB_X_valid=FBpostsXnorm.iloc[validind]
FB_X_test=FBpostsXnorm.iloc[testind]
FB_Y_train=FBpostsY.iloc[trainind]
FB_Y_valid=FBpostsY.iloc[validind]
FB_Y_test=FBpostsY.iloc[testind]


In [149]:
FB_X_train.describe()

,account_idnorm,Categorynorm,date_delta,share_count,reaction_count,comment_count
count,1939.000000,1939.000000,1939.000000,1939.000000,1939.000000,1939.000000
mean,0.001068,0.002823,3.820010,-0.007252,0.000431,0.003752
std,0.998766,1.000099,2.864518,0.991106,0.990227,0.983465
min,-1.803096,-0.995968,0.000000,-1.800733,-2.659409,-2.881335
25%,0.519541,-0.995968,1.000000,-0.758386,-0.718857,-0.671639
50%,0.519541,-0.995968,3.000000,-0.168551,-0.084148,0.048060
75%,0.594356,0.964042,7.000000,0.698313,0.638486,0.687854
max,0.663625,1.058028,8.000000,3.566592,3.225071,4.293052


In [150]:
FB_Y_train.describe()

count    1939.000000
mean        0.635508
std         0.213493
min         0.000000
25%         0.500000
50%         0.750000
75%         0.750000
max         0.750000
Name: Rating, dtype: float64

In [151]:
FB_Y_test.describe()

count    115.000000
mean       0.593478
std        0.239839
min        0.000000
25%        0.500000
50%        0.750000
75%        0.750000
max        0.750000
Name: Rating, dtype: float64

In [152]:
FB_X_train.to_pickle("FB_X_train.pkl") 
FB_X_valid.to_pickle("FB_X_valid.pkl") 
FB_X_test.to_pickle("FB_X_test.pkl") 
FB_Y_train.to_pickle("FB_Y_train.pkl") 
FB_Y_valid.to_pickle("FB_Y_valid.pkl") 
FB_Y_test.to_pickle("FB_Y_test.pkl") 


In [156]:
np.save("FB_X_train.npy", FB_X_train.values) 
np.save("FB_X_valid.npy", FB_X_valid.values)
np.save("FB_X_test.npy", FB_X_test.values)
np.save("FB_Y_train.npy", FB_Y_train.values)
np.save("FB_Y_valid.npy", FB_Y_valid.values)
np.save("FB_Y_test.npy", FB_Y_test.values)

In [159]:
FB_X_train.values.shape

(1939, 6)